In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
net = model_lib.MultiHGModel()

pretrained_dict = torch.load(model_dir+"low_02.pt")
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        in_file = "./results/"+str(i)+".png"
        out_file = "./results/"+str(i)+"_out.png"
        a,b,c,d= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy(),batch_bboxes.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        d = np.moveaxis(d,1,-1)
        
        font = ImageFont.truetype("./results/Aaargh.ttf",14) 
        
        im = (a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255
        im[:,:,0] = np.where(b[0][:,:,0]>0,255,im[:,:,0])
        im[:,:,1] = np.where(b[0][:,:,0]>0,255,im[:,:,1])
        im[:,:,2] = np.where(b[0][:,:,0]>0,255,im[:,:,2])
        
        im = Image.fromarray(im.astype(np.uint8),"RGB")
        draw = ImageDraw.Draw(im)
        draw.rectangle(((0, 0), (40, 20)), fill="black")
        draw.text((5, 5), config.CLASS_NAMES[np.argmax(batch_one_hot.numpy())], font=font,fill=(255,255,255))
        im.resize((224,224),Image.BILINEAR).save(in_file, "PNG")
        
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_bboxes.cuda(),batch_one_hot.cuda()
        pred_class,pred_masks = net([batch_images,batch_impulses])
        
        pred_mask = pred_masks[1]
        pred_mask = F.threshold(F.sigmoid(pred_mask),0.7,0)
        pred_mask = pred_mask.squeeze().cpu().numpy()
        pred_mask = Image.fromarray((pred_mask*255).astype(np.uint8),"L").resize((224,224),Image.BICUBIC)
        pred_mask = np.array(pred_mask)
        img = (a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255
        img[:,:,0] = np.where(pred_mask>0,pred_mask,img[:,:,0])
        im = Image.fromarray(img.astype(np.uint8),"RGB").resize((224,224),Image.BILINEAR)
        
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        pred_class = indices[0]
        
        draw = ImageDraw.Draw(im)
        draw.rectangle(((0, 0), (40, 20)), fill="black")
        draw.text((5, 5), config.CLASS_NAMES[int(pred_class)], font=font,fill=(255,255,255))
        im.save(out_file, "PNG")

        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         input()

tensor(0.7276, device='cuda:0') tensor(49, device='cuda:0') sandwich
tensor(0.1442, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(1.00000e-02 *
       7.6670, device='cuda:0') tensor(43, device='cuda:0') fork
tensor(1.00000e-02 *
       2.2519, device='cuda:0') tensor(44, device='cuda:0') knife
tensor(1.00000e-03 *
       8.4323, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(0.4048, device='cuda:0') tensor(43, device='cuda:0') fork
tensor(0.3325, device='cuda:0') tensor(44, device='cuda:0') knife
tensor(0.2570, device='cuda:0') tensor(45, device='cuda:0') spoon
tensor(1.00000e-03 *
       1.6778, device='cuda:0') tensor(77, device='cuda:0') scissors
tensor(1.00000e-03 *
       1.5206, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(0.9996, device='cuda:0') tensor(33, device='cuda:0') sports ball
tensor(1.00000e-04 *
       1.5227, device='cuda:0') tensor(30, device='cuda:0') frisbee
tensor(1.00000e-04 *
       1.0427, device='cuda:0') tensor(48,

tensor(0.6366, device='cuda:0') tensor(10, device='cuda:0') traffic light
tensor(0.1568, device='cuda:0') tensor(2, device='cuda:0') bicycle
tensor(0.1189, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(1.00000e-02 *
       1.7815, device='cuda:0') tensor(13, device='cuda:0') parking meter
tensor(1.00000e-02 *
       1.5373, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(0.2830, device='cuda:0') tensor(38, device='cuda:0') surfboard
tensor(0.2299, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1947, device='cuda:0') tensor(32, device='cuda:0') snowboard
tensor(0.1381, device='cuda:0') tensor(34, device='cuda:0') kite
tensor(1.00000e-02 *
       5.2861, device='cuda:0') tensor(31, device='cuda:0') skis
tensor(0.9930, device='cuda:0') tensor(75, device='cuda:0') clock
tensor(1.00000e-03 *
       3.2316, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(1.00000e-03 *
       1.0008, device='cuda:0') tensor(10, device='cuda:0') traffic 

tensor(0.5746, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.1611, device='cuda:0') tensor(14, device='cuda:0') bench
tensor(1.00000e-02 *
       8.1949, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       7.7072, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-02 *
       4.6155, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(0.9923, device='cuda:0') tensor(5, device='cuda:0') airplane
tensor(1.00000e-03 *
       3.7299, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-03 *
       1.3196, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-04 *
       6.9374, device='cuda:0') tensor(3, device='cuda:0') car
tensor(1.00000e-04 *
       5.2889, device='cuda:0') tensor(9, device='cuda:0') boat
tensor(0.9964, device='cuda:0') tensor(23, device='cuda:0') zebra
tensor(1.00000e-03 *
       2.8994, device='cuda:0') tensor(36, device='cuda:0') baseball glove
tensor(1.00000e-04 *
       4.8570,

tensor(0.9970, device='cuda:0') tensor(28, device='cuda:0') tie
tensor(1.00000e-03 *
       1.2752, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-03 *
       1.2230, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-04 *
       2.4935, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-04 *
       1.4119, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(0.6729, device='cuda:0') tensor(22, device='cuda:0') bear
tensor(0.2908, device='cuda:0') tensor(21, device='cuda:0') elephant
tensor(1.00000e-02 *
       1.0085, device='cuda:0') tensor(20, device='cuda:0') cow
tensor(1.00000e-03 *
       8.2132, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-03 *
       5.1861, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(0.9988, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(1.00000e-03 *
       1.0736, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-04 *
       1.12

tensor(0.9981, device='cuda:0') tensor(23, device='cuda:0') zebra
tensor(1.00000e-03 *
       1.6888, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-04 *
       1.3327, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-05 *
       4.8489, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-05 *
       2.9614, device='cuda:0') tensor(16, device='cuda:0') cat
tensor(0.4272, device='cuda:0') tensor(36, device='cuda:0') baseball glove
tensor(0.3537, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       7.3390, device='cuda:0') tensor(28, device='cuda:0') tie
tensor(1.00000e-02 *
       6.6546, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       3.5519, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(0.4080, device='cuda:0') tensor(41, device='cuda:0') wine glass
tensor(0.2494, device='cuda:0') tensor(45, device='cuda:0') spoon
tensor(0.1368, device='cuda:0') tensor(42, device='

...

tensor(0.2158, device='cuda:0') tensor(47, device='cuda:0') banana
tensor(0.1296, device='cuda:0') tensor(38, device='cuda:0') surfboard
tensor(0.1090, device='cuda:0') tensor(37, device='cuda:0') skateboard
tensor(0.1077, device='cuda:0') tensor(36, device='cuda:0') baseball glove
tensor(1.00000e-02 *
       7.4587, device='cuda:0') tensor(30, device='cuda:0') frisbee
tensor(1.0000, device='cuda:0') tensor(12, device='cuda:0') stop sign
tensor(1.00000e-06 *
       2.2794, device='cuda:0') tensor(13, device='cuda:0') parking meter
tensor(1.00000e-07 *
       7.1663, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-07 *
       6.9503, device='cuda:0') tensor(10, device='cuda:0') traffic light
tensor(1.00000e-08 *
       7.2966, device='cuda:0') tensor(75, device='cuda:0') clock
tensor(0.9995, device='cuda:0') tensor(13, device='cuda:0') parking meter
tensor(1.00000e-04 *
       2.3708, device='cuda:0') tensor(10, device='cuda:0') traffic light
tensor(1.00000e-04 *
       1

tensor(0.9339, device='cuda:0') tensor(37, device='cuda:0') skateboard
tensor(1.00000e-02 *
       2.0802, device='cuda:0') tensor(4, device='cuda:0') motorcycle
tensor(1.00000e-02 *
       1.3414, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       1.0178, device='cuda:0') tensor(36, device='cuda:0') baseball glove
tensor(1.00000e-03 *
       6.1977, device='cuda:0') tensor(32, device='cuda:0') snowboard
tensor(0.9446, device='cuda:0') tensor(45, device='cuda:0') spoon
tensor(1.00000e-02 *
       1.8131, device='cuda:0') tensor(43, device='cuda:0') fork
tensor(1.00000e-02 *
       1.2869, device='cuda:0') tensor(41, device='cuda:0') wine glass
tensor(1.00000e-03 *
       9.4184, device='cuda:0') tensor(44, device='cuda:0') knife
tensor(1.00000e-03 *
       3.9184, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(0.6251, device='cuda:0') tensor(20, device='cuda:0') cow
tensor(0.3557, device='cuda:0') tensor(18, device='cuda:0') horse
tensor(1.00000e-

tensor(0.8834, device='cuda:0') tensor(74, device='cuda:0') book
tensor(1.00000e-02 *
       6.7518, device='cuda:0') tensor(64, device='cuda:0') laptop
tensor(1.00000e-02 *
       1.2322, device='cuda:0') tensor(60, device='cuda:0') bed
tensor(1.00000e-03 *
       6.1234, device='cuda:0') tensor(29, device='cuda:0') suitcase
tensor(1.00000e-03 *
       5.1944, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(0.9789, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       1.3628, device='cuda:0') tensor(25, device='cuda:0') backpack
tensor(1.00000e-03 *
       2.9031, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-03 *
       1.0082, device='cuda:0') tensor(4, device='cuda:0') motorcycle
tensor(1.00000e-04 *
       9.7859, device='cuda:0') tensor(14, device='cuda:0') bench
tensor(0.9972, device='cuda:0') tensor(19, device='cuda:0') sheep
tensor(1.00000e-03 *
       1.6716, device='cuda:0') tensor(22, device='cuda:0') bear
tenso

tensor(0.9998, device='cuda:0') tensor(65, device='cuda:0') mouse
tensor(1.00000e-05 *
       4.3468, device='cuda:0') tensor(45, device='cuda:0') spoon
tensor(1.00000e-05 *
       3.7212, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-05 *
       3.4064, device='cuda:0') tensor(68, device='cuda:0') cell phone
tensor(1.00000e-05 *
       3.1563, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(0.6130, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(0.3588, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(1.00000e-02 *
       1.1338, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-03 *
       7.5379, device='cuda:0') tensor(61, device='cuda:0') dining table
tensor(1.00000e-03 *
       3.5985, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(0.9368, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       4.1392, device='cuda:0') tensor(14, device='cuda:0') bench
tensor(1.00000e-02 *
   

tensor(0.7079, device='cuda:0') tensor(72, device='cuda:0') sink
tensor(1.00000e-02 *
       7.8940, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(1.00000e-02 *
       6.6731, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       3.4482, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(1.00000e-02 *
       2.2150, device='cuda:0') tensor(62, device='cuda:0') toilet
tensor(0.9679, device='cuda:0') tensor(0, device='cuda:0') BG
tensor(1.00000e-03 *
       5.6971, device='cuda:0') tensor(14, device='cuda:0') bench
tensor(1.00000e-03 *
       5.2385, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-03 *
       4.4303, device='cuda:0') tensor(60, device='cuda:0') bed
tensor(1.00000e-03 *
       4.0727, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(0.2148, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(0.1518, device='cuda:0') tensor(57, device='cuda:0') chair
tensor(1.00000e-02 *
       8.3187, device='cud